In [1]:
%%html
<img src='./picture/CNN7.jpg', width=800>

In [2]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd

In [3]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3801,))
])

In [4]:
train_dataset = datasets.MNIST(root='./dataset/mnist/',
                               train=True,
                               download=False,
                               transform=transform)

train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=batch_size)

In [5]:
test_dataset = datasets.MNIST(root='./dataset/mnist/',
                              train=False,
                              download=False,
                              transform=transform)

test_loader = DataLoader(test_dataset,
                         shuffle=False,
                         batch_size=batch_size)

In [6]:
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)
        
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        
        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)
        
        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)        
        self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)
        
    def forward(self, x):
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        
        outputs = [branch_pool, branch1x1, branch5x5, branch3x3]
        return torch.cat(outputs, dim=1)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=3)
        
        self.incep1 = InceptionA(10)
        self.incep2 = InceptionA(20)
        
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(2200, 10)
    
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incep1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size, -1)
        return self.fc(x)

In [8]:
model = Net()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(88, 20, kernel_size=(3, 3), stride=(1, 1))
  (incep1): InceptionA(
    (branch_pool): Conv2d(10, 24, kernel_size=(1, 1), stride=(1, 1))
    (branch1x1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch3x3_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch3x3_3): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (incep2): InceptionA(
    (branch_pool): Conv2d(20, 24, kernel_size=(1, 1), stride=(1, 1))
    (branch1x1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
   

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [10]:
def train(epoch):
    run_loss = 0
    for batch_idx, [inputs, labels] in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        run_loss += loss.item()
        
        temp = pd.Series([(epoch + 1), (batch_idx + 1), loss.item()], index=['epoch', 'batch_idx', 'loss'])
        global log # 声明引用的是全局变量
        log = log.append(temp, ignore_index=True)
        log.to_csv('log/log1.csv', index=False)
        
        if batch_idx % 300 == 299:
            print('[%d, %5d], loss: %.3f' % (epoch + 1, batch_idx + 1, run_loss / 300))
            run_loss = 0

In [11]:
def test():
    correct = 0;
    total = 0;
    with torch.no_grad():
        for [inputs, labels] in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, pred = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
    print('Accuracy on test set: %d %% [%d/%d] ' % (100 * correct / total, correct, total))

In [12]:
if __name__ == '__main__':
    log = pd.DataFrame(index=[], columns=['epoch', 'batch_idx', 'loss'])
    for epoch in range(10):
        train(epoch)
        test()

[1,   300], loss: 1.404
[1,   600], loss: 0.337
[1,   900], loss: 0.196
Accuracy on test set: 96 % [9608/10000] 
[2,   300], loss: 0.134
[2,   600], loss: 0.109
[2,   900], loss: 0.097
Accuracy on test set: 97 % [9765/10000] 
[3,   300], loss: 0.083
[3,   600], loss: 0.085
[3,   900], loss: 0.078
Accuracy on test set: 97 % [9787/10000] 
[4,   300], loss: 0.072
[4,   600], loss: 0.071
[4,   900], loss: 0.066
Accuracy on test set: 98 % [9830/10000] 
[5,   300], loss: 0.064
[5,   600], loss: 0.062
[5,   900], loss: 0.063
Accuracy on test set: 98 % [9808/10000] 
[6,   300], loss: 0.060
[6,   600], loss: 0.051
[6,   900], loss: 0.058
Accuracy on test set: 98 % [9823/10000] 
[7,   300], loss: 0.049
[7,   600], loss: 0.055
[7,   900], loss: 0.053
Accuracy on test set: 98 % [9831/10000] 
[8,   300], loss: 0.048
[8,   600], loss: 0.052
[8,   900], loss: 0.051
Accuracy on test set: 98 % [9858/10000] 
[9,   300], loss: 0.044
[9,   600], loss: 0.045
[9,   900], loss: 0.049
Accuracy on test set: 98